In [2]:
from pyspark.sql import SparkSession

MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("sql-query")\
    .config("spark.executor.memory", MAX_MEMORY)\
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()
    
sc = spark.sparkContext

22/10/11 13:39:36 WARN Utils: Your hostname, gimgeon-uui-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.16 instead (on interface en0)
22/10/11 13:39:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/11 13:39:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
event_parquet_directory = "/Users/kuno/code/iag/parquet_data/event"
event_df = spark.read.parquet(f"{event_parquet_directory}")

event_df.show(10)

attribution_parquet_directory = "/Users/kuno/code/iag/parquet_data/attribution"
attribution_df = spark.read.parquet(f"{attribution_parquet_directory}")

attribution_df.show(10)

+-------------+---+-----+-------+---------------+--------------------+--------------------+--------+-----+
|identity_adid| os|model|country|     event_name|              log_id|     server_datetime|quantity|price|
+-------------+---+-----+-------+---------------+--------------------+--------------------+--------+-----+
|   1028207716|8.9|  8.9|     kr|  custom:battle|ec27d5d4-590a-477...|2018-06-04 08:49:...|    null| null|
|    722905290|7.1|  7.1|     kr|  custom:battle|1368cfed-ce95-411...|2018-06-04 17:56:...|    null| null|
|   1514811938|3.4|  3.4|     kr|  custom:battle|bf5a0afd-4478-4cf...|2018-06-04 17:59:...|    null| null|
|     52925716|6.0|  6.0|     ge|  custom:battle|489d4bf9-1a30-437...|2018-06-04 05:05:...|    null| null|
|    620228434|7.1|  7.1|     gb|      abx:login|5231a92e-fa13-4ef...|2018-06-04 05:06:...|    null| null|
|    210821615|6.0|  6.0|     us|  abx:firstopen|d4df8648-6b9f-472...|2018-06-04 04:47:...|    null| null|
|   1161490770|8.9|  8.9|     us|abx:

In [4]:
event_df.dtypes
attribution_df.dtypes

[('partner', 'string'),
 ('campaign', 'string'),
 ('server_datetime', 'string'),
 ('tracker_id', 'string'),
 ('log_id', 'string'),
 ('attribution_type', 'string'),
 ('identity_adid', 'string')]

In [5]:
event_df.createOrReplaceTempView("event")
attribution_df.createOrReplaceTempView("attribution")

In [6]:
query_event = "SELECT * FROM event"
query_attribution = "SELECT * FROM attribution"

spark.sql(query_event).show(5)
spark.sql(query_attribution).show(5)

+-------------+---+-----+-------+-------------+--------------------+--------------------+--------+-----+
|identity_adid| os|model|country|   event_name|              log_id|     server_datetime|quantity|price|
+-------------+---+-----+-------+-------------+--------------------+--------------------+--------+-----+
|   1028207716|8.9|  8.9|     kr|custom:battle|ec27d5d4-590a-477...|2018-06-04 08:49:...|    null| null|
|    722905290|7.1|  7.1|     kr|custom:battle|1368cfed-ce95-411...|2018-06-04 17:56:...|    null| null|
|   1514811938|3.4|  3.4|     kr|custom:battle|bf5a0afd-4478-4cf...|2018-06-04 17:59:...|    null| null|
|     52925716|6.0|  6.0|     ge|custom:battle|489d4bf9-1a30-437...|2018-06-04 05:05:...|    null| null|
|    620228434|7.1|  7.1|     gb|    abx:login|5231a92e-fa13-4ef...|2018-06-04 05:06:...|    null| null|
+-------------+---+-----+-------+-------------+--------------------+--------------------+--------+-----+
only showing top 5 rows

+-------+--------+------------

In [27]:
# 일별 Active User 데이터 추출

query = """
SELECT server_datetime AS Date, 
       COUNT(DISTINCT identity_adid) AS ActiveUser
FROM (SELECT CAST(server_datetime AS DATE), 
             identity_adid 
      FROM event
      WHERE server_datetime BETWEEN '2018-05-01 00:00:00' 
                                AND '2018-06-13 23:59:59')
GROUP BY server_datetime
ORDER BY server_datetime;
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-1")


+----------+----------+
|      Date|ActiveUser|
+----------+----------+
|2018-05-01|      2830|
|2018-05-02|    333893|
|2018-05-03|    341539|
|2018-05-04|    350451|
|2018-05-05|    350957|
|2018-05-06|    351161|
|2018-05-07|    347310|
|2018-05-08|    347407|
|2018-05-09|    286317|
|2018-05-10|    318275|
|2018-05-11|    163929|
|2018-05-12|    347605|
|2018-05-13|    348300|
|2018-05-14|    131192|
|2018-05-15|     72188|
|2018-05-16|     73165|
|2018-05-17|     41765|
|2018-05-18|     26393|
|2018-05-20|       826|
|2018-05-21|    138633|
+----------+----------+
only showing top 20 rows



In [33]:
# 일별 구매 유저 및 구매 금액 추출

query = """
SELECT server_datetime AS Date, 
       COUNT(DISTINCT identity_adid) AS ActiveUser, 
       SUM(price) AS PurchaseAmount
FROM (SELECT CAST(server_datetime AS DATE), 
             identity_adid, 
             price
      FROM event
      WHERE (event_name = 'abx:purchase') 
        AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                 AND '2018-06-13 23:59:59'))
GROUP BY server_datetime
ORDER BY server_datetime;
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-2")

+----------+----------+--------------+
|      Date|ActiveUser|PurchaseAmount|
+----------+----------+--------------+
|2018-05-01|        47|      118000.0|
|2018-05-02|      6233|      1.5529E7|
|2018-05-03|      6314|      1.5742E7|
|2018-05-04|      6397|      1.6024E7|
|2018-05-05|      6429|      1.6088E7|
|2018-05-06|      6458|      1.6138E7|
|2018-05-07|      6530|      1.6327E7|
|2018-05-08|      6574|      1.6472E7|
|2018-05-09|      5313|      1.3306E7|
|2018-05-10|      5889|      1.4677E7|
|2018-05-11|      3045|     7623000.0|
|2018-05-12|      6398|      1.5923E7|
|2018-05-13|      6303|      1.5782E7|
|2018-05-14|     22778|     1.03443E8|
|2018-05-15|     31179|     1.46936E8|
|2018-05-16|     31490|     1.47298E8|
|2018-05-17|     18097|     1.84703E8|
|2018-05-18|     10222|     1.08295E8|
|2018-05-20|        52|      130000.0|
|2018-05-21|     56051|     2.71162E8|
+----------+----------+--------------+
only showing top 20 rows



In [34]:
# 이벤트별 User 데이터 추출

query = """
SELECT event_name AS event,
       COUNT(server_datetime) AS ActiveUser
FROM (SELECT CAST(server_datetime AS DATE),
             event_name
      FROM event
      WHERE (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                 AND '2018-06-13 23:59:59'))
GROUP BY event_name;
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-3")

+-----------------+----------+
|            event|ActiveUser|
+-----------------+----------+
|     abx:purchase|   1131010|
|abx:start_session|   3220266|
|        abx:login|   2757284|
|    abx:firstopen|   3679514|
|    custom:battle|   4841266|
|  abx:end_session|   2335643|
+-----------------+----------+



In [35]:
# 캠페인별 User 데이터 추출

query = """
SELECT campaign AS campaign_id,
       COUNT(identity_adid) AS ActiveUser
FROM (SELECT DISTINCT CAST(server_datetime AS DATE),
             campaign,
             identity_adid
      FROM attribution
      WHERE (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                 AND '2018-06-13 23:59:59')
        AND (NOT campaign IS NULL))
GROUP BY campaign
ORDER BY campaign;
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-4")

+-----------+----------+
|campaign_id|ActiveUser|
+-----------+----------+
|  campaign1|    277843|
|  campaign2|    342451|
|  campaign3|     75629|
+-----------+----------+



In [36]:
# 캠페인별 구매 금액 추출

query = """
SELECT atsub.campaign AS campaign_id,
       COUNT(atsub.identity_adid) AS ActiveUser,
       SUM(evsub.PersonalPrice) AS PurchaseAmount
FROM (SELECT DISTINCT CAST(server_datetime AS DATE) AS Date,
                      campaign,
                      identity_adid
      FROM attribution
      WHERE (NOT campaign IS NULL)
        AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                 AND '2018-06-13 23:59:59')) atsub
JOIN (SELECT CAST(server_datetime AS DATE) AS Date, 
             identity_adid, 
             SUM(price) AS PersonalPrice
      FROM event
      WHERE (event_name = 'abx:purchase') 
        AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                 AND '2018-06-13 23:59:59')
      GROUP BY server_datetime, identity_adid) evsub
ON (atsub.Date = evsub.Date) AND (atsub.identity_adid = evsub.identity_adid)
GROUP BY atsub.campaign
ORDER BY atsub.campaign;
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-5")

+-----------+----------+--------------+
|campaign_id|ActiveUser|PurchaseAmount|
+-----------+----------+--------------+
|  campaign1|    370130|      9.2504E8|
|  campaign2|    405550|    1.012998E9|
|  campaign3|    180950|     4.52081E8|
+-----------+----------+--------------+



In [37]:
# 국가, 캠페인별 구매 금액 추출

query = """
SELECT country AS country,
       atsub.campaign AS campaign_id,
       COUNT(atsub.identity_adid) AS ActiveUser,
       SUM(evsub.PersonalPrice) AS PurchaseAmount
FROM (SELECT DISTINCT CAST(server_datetime AS DATE) AS Date,
                      campaign,
                      identity_adid
      FROM attribution
      WHERE (NOT campaign IS NULL)
        AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                 AND '2018-06-13 23:59:59')) atsub
LEFT JOIN (SELECT CAST(server_datetime AS DATE) AS Date, 
             country,
             identity_adid, 
             SUM(price) AS PersonalPrice
      FROM event
      WHERE (event_name = 'abx:purchase') 
        AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                 AND '2018-06-13 23:59:59')
      GROUP BY server_datetime, identity_adid, country) evsub
ON (atsub.Date = evsub.Date) AND (atsub.identity_adid = evsub.identity_adid)
WHERE NOT country IS NULL
GROUP BY country, atsub.campaign
ORDER BY country, atsub.campaign;
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-6")

+-------+-----------+----------+--------------+
|country|campaign_id|ActiveUser|PurchaseAmount|
+-------+-----------+----------+--------------+
|     gb|  campaign1|     73681|     1.84183E8|
|     gb|  campaign2|     80882|     2.02007E8|
|     gb|  campaign3|     35452|      8.8527E7|
|     ge|  campaign1|     36660|      9.1569E7|
|     ge|  campaign2|     40468|     1.00975E8|
|     ge|  campaign3|     18435|      4.5971E7|
|     jp|  campaign1|     75018|     1.87545E8|
|     jp|  campaign2|     81890|       2.047E8|
|     jp|  campaign3|     36591|      9.1427E7|
|     kr|  campaign1|    111304|     2.78094E8|
|     kr|  campaign2|    121954|     3.04629E8|
|     kr|  campaign3|     54539|     1.36259E8|
|     us|  campaign1|     73467|     1.83649E8|
|     us|  campaign2|     80356|     2.00687E8|
|     us|  campaign3|     35933|      8.9897E7|
+-------+-----------+----------+--------------+



In [12]:
# Funnel 데이터 추출

query = """
SELECT *
FROM (SELECT server_datetime,
             event_name AS event_case,
             COUNT(identity_adid) AS ActiveUser
      FROM (SELECT CAST(server_datetime AS DATE),
                   event_name,
                   identity_adid
            FROM event
            WHERE (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                       AND '2018-06-13 23:59:59'))
      GROUP BY server_datetime, event_name)
PIVOT (
    MAX(ActiveUser)
    FOR event_case IN ('abx:firstopen', 'abx:login', 'abx:purchase'))
ORDER BY server_datetime;
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-7")

+---------------+-------------+---------+------------+
|server_datetime|abx:firstopen|abx:login|abx:purchase|
+---------------+-------------+---------+------------+
|     2018-05-01|         1387|      386|          47|
|     2018-05-02|       161986|    44707|        6233|
|     2018-05-03|       166300|    45264|        6314|
|     2018-05-04|       170683|    46182|        6397|
|     2018-05-05|       170134|    46467|        6429|
|     2018-05-06|       169761|    46888|        6458|
|     2018-05-07|       168096|    45991|        6530|
|     2018-05-08|       168138|    46332|        6574|
|     2018-05-09|       139696|    38361|        5314|
|     2018-05-10|       154202|    42384|        5889|
+---------------+-------------+---------+------------+



In [38]:
# Funnel 데이터 추출

query = """
SELECT Date,
       count(distinct case when event_name = 'abx:firstopen' 
         then identity_adid else null end) AS firstopen,
       count(distinct case when event_name = 'abx:login' 
         then identity_adid else null end) AS login,
       count(distinct case when event_name = 'abx:purchase' 
         then identity_adid else null end) AS purchase
    
FROM (SELECT CAST(server_datetime AS DATE) AS Date,
             event_name,
             identity_adid
      FROM event
      WHERE (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                 AND '2018-06-13 23:59:59'))
WHERE event_name in ('abx:firstopen', 'abx:login', 'abx:purchase')
GROUP BY Date
ORDER BY Date
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-7")

+----------+---------+------+--------+
|      Date|firstopen| login|purchase|
+----------+---------+------+--------+
|2018-05-01|     1387|   386|      47|
|2018-05-02|   161986| 44707|    6233|
|2018-05-03|   166300| 45264|    6314|
|2018-05-04|   170683| 46182|    6397|
|2018-05-05|   170134| 46467|    6429|
|2018-05-06|   169761| 46888|    6458|
|2018-05-07|   168096| 45991|    6530|
|2018-05-08|   168138| 46332|    6574|
|2018-05-09|   139696| 38361|    5313|
|2018-05-10|   154202| 42384|    5889|
|2018-05-11|    79659| 22031|    3045|
|2018-05-12|   169346| 46352|    6398|
|2018-05-13|   169504| 46229|    6303|
|2018-05-14|    89409| 60316|   22778|
|2018-05-15|    72049| 72133|   31179|
|2018-05-16|    73041| 73114|   31490|
|2018-05-17|    41630| 41713|   18097|
|2018-05-18|    26375| 25607|   10222|
|2018-05-20|      161|   143|      52|
|2018-05-21|    84913|115679|   56051|
+----------+---------+------+--------+
only showing top 20 rows



In [39]:
# 파트너별 new_install 유저의 funnel 데이터 추출

query = """
SELECT partner,
       count(attribution_type) AS newinstalluser,
       count(distinct case when event_name = 'abx:firstopen' 
         then evsub.identity_adid else null end) AS firstopen,
       count(distinct case when event_name = 'abx:login' 
         then evsub.identity_adid else null end) AS login,
       count(distinct case when event_name = 'abx:purchase' 
         then evsub.identity_adid else null end) AS purchase
FROM (SELECT event_name,
             identity_adid
      FROM event
      WHERE (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                 AND '2018-06-13 23:59:59')) evsub
RIGHT jOIN (SELECT partner, 
                   identity_adid,
                   attribution_type
            FROM attribution
            WHERE NOT partner IS NULL
              AND attribution_type = 0
              AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                       AND '2018-06-13 23:59:59')) atsub
ON evsub.identity_adid = atsub.identity_adid
GROUP BY partner    

"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-8")

+--------------------+--------------+---------+------+--------+
|             partner|newinstalluser|firstopen| login|purchase|
+--------------------+--------------+---------+------+--------+
|8MCosUQMik2Muvd-M...|       4045059|   533793|258806|   98521|
+--------------------+--------------+---------+------+--------+



In [ ]:
# 일자별 리텐션
query = """
SELECT re.Date, 
       COUNT(DISTINCT re.identity_adid), 
       COUNT(DISTINCT re1.identity_adid), 
       COUNT(DISTINCT re2.identity_adid),
       COUNT(DISTINCT re3.identity_adid),
       COUNT(DISTINCT re7.identity_adid)   
FROM (SELECT CAST(server_datetime AS DATE) AS Date
             , identity_adid
      FROM event
      WHERE event_name = 'abx:firstopen'
      AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                               AND '2018-06-10 23:59:59')
      ORDER BY Date) re
LEFT JOIN (SELECT CAST(server_datetime AS DATE) AS Date
                  , identity_adid
           FROM event
           WHERE event_name = 'abx:firstopen'
           AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                    AND '2018-06-10 23:59:59')
           ORDER BY Date) re1
ON (re.identity_adid = re1.identity_adid) and (re.Date = re1.Date + (INTERVAL '-1' DAY))
LEFT JOIN (SELECT CAST(server_datetime AS DATE) AS Date
                  , identity_adid
           FROM event
           WHERE event_name = 'abx:firstopen'
           AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                    AND '2018-06-10 23:59:59')
           ORDER BY Date) re2
ON (re.identity_adid = re2.identity_adid) and (re.Date = re2.Date + (INTERVAL '-2' DAY))
LEFT JOIN (SELECT CAST(server_datetime AS DATE) AS Date
                  , identity_adid
           FROM event
           WHERE event_name = 'abx:firstopen'
           AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                    AND '2018-06-10 23:59:59')
           ORDER BY Date) re3
ON (re.identity_adid = re3.identity_adid) and (re.Date = re3.Date + (INTERVAL '-3' DAY))
LEFT JOIN (SELECT CAST(server_datetime AS DATE) AS Date
                  , identity_adid
           FROM event
           WHERE event_name = 'abx:firstopen'
           AND (server_datetime BETWEEN '2018-05-01 00:00:00' 
                                    AND '2018-06-10 23:59:59')
           ORDER BY Date) re7
ON (re.identity_adid = re7.identity_adid) and (re.Date = re7.Date + (INTERVAL '-7' DAY))
GROUP BY re.Date
;
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-9")

In [ ]:
# 일자별 리텐션
query = """
SELECT DT1.Date, D0, D1, D2, D3, D7
FROM   (SELECT re.Date AS Date, 
               count(distinct re.identity_adid) AS D0,
               count(distinct re1.identity_adid) AS D1
        FROM (SELECT CAST(server_datetime AS DATE) AS Date
                     , identity_adid
              FROM event
              WHERE event_name = 'abx:firstopen'
              AND (server_datetime BETWEEN '2018-05-14 00:00:00' 
                                       AND '2018-06-10 23:59:59')
              ORDER BY Date) re
        LEFT JOIN (SELECT CAST(server_datetime AS DATE) AS Date
                     , identity_adid
                   FROM event
                   WHERE event_name = 'abx:firstopen'
                   AND (server_datetime BETWEEN '2018-05-14 00:00:00' 
                                            AND '2018-06-10 23:59:59')
                   ORDER BY Date) re1
        ON (re.identity_adid = re1.identity_adid) and (re.Date = re1.Date + (INTERVAL '-1' DAY))
        GROUP BY re.Date) DT1
LEFT JOIN (SELECT re.Date AS Date, 
                  count(distinct re2.identity_adid) AS D2
           FROM (SELECT CAST(server_datetime AS DATE) AS Date
                        , identity_adid
                 FROM event
                 WHERE event_name = 'abx:firstopen'
                 AND (server_datetime BETWEEN '2018-05-14 00:00:00' 
                                          AND '2018-06-10 23:59:59')
                 ORDER BY Date) re
           LEFT JOIN (SELECT CAST(server_datetime AS DATE) AS Date
                        , identity_adid
                      FROM event
                      WHERE event_name = 'abx:firstopen'
                      AND (server_datetime BETWEEN '2018-05-14 00:00:00' 
                                               AND '2018-06-10 23:59:59')
                      ORDER BY Date) re2
           ON (re.identity_adid = re2.identity_adid) and (re.Date = re2.Date + (INTERVAL '-2' DAY))
           GROUP BY re.Date) DT2
ON DT1.Date = DT2.Date
LEFT JOIN (SELECT re.Date AS Date, 
                  count(distinct re3.identity_adid) AS D3
           FROM (SELECT CAST(server_datetime AS DATE) AS Date
                        , identity_adid
                 FROM event
                 WHERE event_name = 'abx:firstopen'
                 AND (server_datetime BETWEEN '2018-05-14 00:00:00' 
                                          AND '2018-06-10 23:59:59')
                 ORDER BY Date) re
           LEFT JOIN (SELECT CAST(server_datetime AS DATE) AS Date
                        , identity_adid
                      FROM event
                      WHERE event_name = 'abx:firstopen'
                      AND (server_datetime BETWEEN '2018-05-14 00:00:00' 
                                               AND '2018-06-10 23:59:59')
                      ORDER BY Date) re3
           ON (re.identity_adid = re3.identity_adid) and (re.Date = re3.Date + (INTERVAL '-3' DAY))
           GROUP BY re.Date) DT3
ON DT1.Date = DT3.Date
LEFT JOIN (SELECT re.Date AS Date, 
                  count(distinct re7.identity_adid) AS D7
           FROM (SELECT CAST(server_datetime AS DATE) AS Date
                        , identity_adid
                 FROM event
                 WHERE event_name = 'abx:firstopen'
                 AND (server_datetime BETWEEN '2018-05-14 00:00:00' 
                                          AND '2018-06-10 23:59:59')
                 ORDER BY Date) re
           LEFT JOIN (SELECT CAST(server_datetime AS DATE) AS Date
                        , identity_adid
                      FROM event
                      WHERE event_name = 'abx:firstopen'
                      AND (server_datetime BETWEEN '2018-05-14 00:00:00' 
                                               AND '2018-06-10 23:59:59')
                      ORDER BY Date) re7
           ON (re.identity_adid = re7.identity_adid) and (re.Date = re7.Date + (INTERVAL '-7' DAY))
           GROUP BY re.Date) DT7
ON DT1.Date = DT7.Date
ORDER BY DT1.Date
;
"""

df = spark.sql(query)
df.show()
df.write.format("csv").save("/Users/kuno/code/iag/result_csv/result-9")